# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'06-25-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'06-25-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-06-26 04:21:43,33.93911,67.709953,111592,4519,67183.0,39890.0,Afghanistan,286.660045,4.049573
1,NaN,NaN,NaN,Albania,2021-06-26 04:21:43,41.15330,20.168300,132506,2455,129975.0,76.0,Albania,4604.420043,1.852746
2,NaN,NaN,NaN,Algeria,2021-06-26 04:21:43,28.03390,1.659600,137772,3678,95878.0,38216.0,Algeria,314.181809,2.669628
3,NaN,NaN,NaN,Andorra,2021-06-26 04:21:43,42.50630,1.521800,13882,127,13697.0,58.0,Andorra,17966.737850,0.914854
4,NaN,NaN,NaN,Angola,2021-06-26 04:21:43,-11.20270,17.873900,38371,883,32605.0,4883.0,Angola,116.748881,2.301217


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,93288,96531,98734,98734,98734,103902,105749,107957,109532,111592
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132476,132481,132484,132488,132490,132490,132496,132497,132499,132506
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,134458,134840,135219,135586,135821,136294,136679,137049,137403,137772


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,3683,3842,3934,3934,3934,4215,4293,4366,4452,4519
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2454,2454,2454,2454,2454,2454,2455,2455,2455,2455
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3598,3605,3615,3624,3631,3641,3650,3660,3669,3678


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,62397,62698,63426,63875,64401,65071,65565,66102,66799,67183
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,129888,129895,129903,129910,129918,129918,129941,129955,129966,129975
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,93586,93831,94093,94336,94571,94822,95084,95331,95599,95878


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7230,7230,7241,7241,7241,7242,7242,7244,7244,7244
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,21839,21839,21868,21868,21868,21901,21901,21921,21921,21945
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2346,2346,2345,2345,2345,2345,2345,2344,2344,2344


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,113,113,113,113,113,113,113,113,113,113
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,313,313,314,314,314,314,314,314,314,314
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,59,59,59,59,59,59,59,60,60,60


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
656,1001.0,Autauga,Alabama,US,2021-06-26 04:21:43,32.539527,-86.644082,7244,113,NaN,NaN,"Autauga, Alabama, US",12966.045571,1.559912
693,1075.0,Lamar,Alabama,US,2021-06-26 04:21:43,33.779950,-88.096680,1472,37,NaN,NaN,"Lamar, Alabama, US",10662.803332,2.513587
694,1077.0,Lauderdale,Alabama,US,2021-06-26 04:21:43,34.901719,-87.656247,9632,250,NaN,NaN,"Lauderdale, Alabama, US",10387.257492,2.595515


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,33603148,603527,0.0
India,30134445,393310,29128267.0
Brazil,18322760,511142,16097711.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,33603148,603527,0.0,32999621.0,2021-06-26 04:21:43,37.950547,-91.419547
India,30134445,393310,29128267.0,612868.0,2021-06-26 04:21:43,23.088275,81.806127
Brazil,18322760,511142,16097711.0,1713907.0,2021-06-26 04:21:43,-12.669522,-48.480493
France,5828266,111101,404107.0,5313058.0,2021-06-26 04:21:43,6.430808,-34.730285
Turkey,5398878,49473,5261892.0,87513.0,2021-06-26 04:21:43,38.963700,35.243300


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3814010,63536,0.0
Texas,2992812,52246,0.0
Florida,2354416,37555,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3814010,63536,0.0,3750474.0,2021-06-26 04:21:43,37.843962,-120.728594
Texas,2992812,52246,0.0,2940566.0,2021-06-26 04:21:43,31.660643,-98.653069
Florida,2354416,37555,0.0,2316861.0,2021-06-26 04:21:43,28.940755,-82.700744
New York,2113398,53644,0.0,2059754.0,2021-06-26 04:21:43,42.544151,-75.474183
Illinois,1390432,25632,0.0,1364800.0,2021-06-26 04:21:43,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1248737,24480,0.0
Arizona,Maricopa,558490,10261,0.0
Illinois,Cook,556715,10478,0.0
Florida,Miami-Dade,506428,6472,0.0
Texas,Harris,403134,6565,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1248737,24480,0.0,1224257.0,2021-06-26 04:21:43,34.308284,-118.228241,6037.0
Arizona,Maricopa,558490,10261,0.0,548229.0,2021-06-26 04:21:43,33.348359,-112.491815,4013.0
Illinois,Cook,556715,10478,0.0,546237.0,2021-06-26 04:21:43,41.841448,-87.816588,17031.0
Florida,Miami-Dade,506428,6472,0.0,499956.0,2021-06-26 04:21:43,25.611236,-80.551706,12086.0
Texas,Harris,403134,6565,0.0,396569.0,2021-06-26 04:21:43,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-23,107957,132497,137049,13873,38002,1263,4326101,224430,30404,649845,...,4683986,360247,107266,4,263372,13989,313015,6898,137026,43480
2021-06-24,109532,132499,137403,13877,38091,1263,4350564,224533,30424,649922,...,4700691,361994,107708,4,265642,14323,313015,6905,140620,44306
2021-06-25,111592,132506,137772,13882,38371,1263,4374587,224635,30457,650016,...,4716065,363580,108184,4,266952,14537,313520,6905,143215,45217


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-23,4366,2455,3660,127,878,42,90986,4503,910,10686,...,128291,5374,722,1,2989,70,3555,1355,1794,1692
2021-06-24,4452,2455,3669,127,881,42,91438,4505,910,10697,...,128312,5413,723,1,3023,72,3555,1356,1855,1709
2021-06-25,4519,2455,3678,127,883,42,91979,4505,910,10700,...,128330,5447,725,1,3038,74,3556,1357,1915,1721


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-23,66102,129955,95331,13688,32322,1221,3944323,216230,23752,636740,...,15705,332112,103074,3,243840,5684,306532,3990,113109,37477
2021-06-24,66799,129966,95599,13694,32340,1221,3967633,216312,23759,636923,...,15712,334971,103472,3,246983,5759,306532,3995,115898,37524
2021-06-25,67183,129975,95878,13697,32605,1221,3990289,216423,23765,637051,...,15718,337748,104007,3,248066,5949,307236,4005,119411,37604


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21,6/21/21,6/22/21,6/23/21,6/24/21,6/25/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7230,7230,7241,7241,7241,7242,7242,7244,7244,7244
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1465,1465,1468,1468,1468,1469,1469,1472,1472,1472
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9596,9596,9603,9603,9603,9612,9612,9621,9621,9632
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,3129,3129,3133,3133,3133,3134,3134,3135,3135,3135
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,16265,16265,16278,16278,16278,16287,16287,16301,16301,16317


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-23,549394,71035,0,891023,346569,3811723,554977,348840,109599,49,...,866608,2989604,412546,24379,3772,679137,448945,163689,677048,61776
2021-06-24,549394,71035,0,891649,346952,3812825,555569,348993,109620,49,...,866670,2991189,413008,24385,3772,679391,449491,163744,677186,61847
2021-06-25,549934,71035,0,892236,347254,3814010,555971,349120,109636,49,...,866670,2992812,413317,24392,3835,679762,449983,163804,677252,61951


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-06-23        7244   21921    2344  2686   6957    1246   2250   14725   
2021-06-24        7244   21921    2344  2686   6957    1246   2250   14725   
2021-06-25        7244   21945    2344  2686   6967    1247   2255   14761   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-23         3731     1875  ...    2898    677     3259      799   
2021-06-24         3731     1875  ...    2902    679     3262      797   
2021-06-25         3733     1874  ...    2902    680     3265      797   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-23           4712  3798  2304          0      927    656  
2021-06-24           4723  3798  2305          0      927    656  
2021-06-25           4729  3801  2315          0      926    656  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-23,11328,374,0,17868,5887,63424,6754,8271,1693,0,...,12539,52183,2336,256,30,11368,5889,2872,8077,740
2021-06-24,11328,374,0,17875,5890,63501,6765,8274,1693,0,...,12543,52217,2337,256,30,11381,5898,2876,8092,740
2021-06-25,11336,374,0,17903,5893,63536,6771,8275,1693,0,...,12543,52246,2351,256,30,11396,5902,2876,8099,740


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-06-23         113     314      60   64    139      42     71     329   
2021-06-24         113     314      60   64    139      42     71     329   
2021-06-25         113     314      60   64    139      42     71     330   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-23          124       45  ...      30     12       31        7   
2021-06-24          124       45  ...      30     12       31        7   
2021-06-25          124       45  ...      30     12       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-23             40    11    13          0       26      6  
2021-06-24             40    11    13          0       26      6  
2021-06-25             40    11    13          0       26      6  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-23,0.020880,0.000008,0.002707,0.000649,0.003380,0.0,0.006355,0.000446,0.000790,0.000180,...,0.003415,0.004982,0.003921,0.0,0.000000,0.015020,0.000000,0.000871,0.025191,0.017933
2021-06-24,0.014589,0.000015,0.002583,0.000288,0.002342,0.0,0.005655,0.000459,0.000658,0.000118,...,0.003566,0.004849,0.004121,0.0,0.008619,0.023876,0.000000,0.001015,0.026229,0.018997
2021-06-25,0.018807,0.000053,0.002686,0.000360,0.007351,0.0,0.005522,0.000454,0.001085,0.000145,...,0.003271,0.004381,0.004419,0.0,0.004931,0.014941,0.001613,0.000000,0.018454,0.020562


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-23,0.017004,0.0,0.002740,0.0,0.003429,0.0,0.007809,0.000222,0.0,0.000187,...,0.000148,0.004486,0.002778,0.0,0.000000,0.014493,0.000000,0.000000,0.028670,0.000591
2021-06-24,0.019698,0.0,0.002459,0.0,0.003417,0.0,0.004968,0.000444,0.0,0.001029,...,0.000164,0.007257,0.001385,0.0,0.011375,0.028571,0.000000,0.000738,0.034002,0.010047
2021-06-25,0.015049,0.0,0.002453,0.0,0.002270,0.0,0.005917,0.000000,0.0,0.000280,...,0.000140,0.006281,0.002766,0.0,0.004962,0.027778,0.000281,0.000737,0.032345,0.007022


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-23,0.008190,0.000108,0.002598,0.001683,0.009968,0.0,0.004056,0.000222,0.000084,0.000341,...,0.005699,0.007432,0.005149,0.0,0.000000,0.024883,0.000000,0.002513,0.011310,0.005069
2021-06-24,0.010544,0.000085,0.002811,0.000438,0.000557,0.0,0.005910,0.000379,0.000295,0.000287,...,0.000446,0.008609,0.003861,0.0,0.012890,0.013195,0.000000,0.001253,0.024658,0.001254
2021-06-25,0.005749,0.000069,0.002918,0.000219,0.008194,0.0,0.005710,0.000513,0.000253,0.000201,...,0.000382,0.008290,0.005170,0.0,0.004385,0.032992,0.002297,0.002503,0.030311,0.002132


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-23,0.000694,0.001015,NaN,0.000550,0.001124,0.000240,0.000909,0.000109,0.000201,0.0,...,0.000218,0.000407,0.001279,0.000328,0.000000,0.000336,0.001792,0.000379,0.000173,0.001167
2021-06-24,0.000000,0.000000,NaN,0.000703,0.001105,0.000289,0.001067,0.000439,0.000192,0.0,...,0.000072,0.000530,0.001120,0.000246,0.000000,0.000374,0.001216,0.000336,0.000204,0.001149
2021-06-25,0.000983,0.000000,NaN,0.000658,0.000870,0.000311,0.000724,0.000364,0.000146,0.0,...,0.000000,0.000543,0.000748,0.000287,0.016702,0.000546,0.001095,0.000366,0.000097,0.001682


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-06-23      0.000276  0.000913 -0.000426  0.000372  0.001007  0.000803   
2021-06-24      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2021-06-25      0.000000  0.001095  0.000000  0.000000  0.001437  0.000803   

Province_State                                          ...  Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...     Park   
2021-06-23      0.001781  0.000068  0.000805  0.001068  ...  0.00000   
2021-06-24      0.000000  0.000000  0.000000  0.000000  ...  0.00138   
2021-06-25      0.002222  0.002445  0.000536 -0.000533  ...  0.00000   

Province_State                                                              \
Admin2            Platte  Sheridan  Sublette Sweetwater    Teton     Uinta   
2021-06-23      0.000000  0.002461 -0.003741   0.002767  0.00000 -0.001733   
2021-06-24      0.002954  0.000921 -0.002503   0.002334  0.00000  0.000434   
2021-06-25      0.001473  0.000920  0.000000   0.001270  0.00079  0.004338   

Province_State                              
Admin2         Unassigned  Washakie Weston  
2021-06-23            NaN  0.000000    0.0  
2021-06-24            NaN  0.000000    0.0  
2021-06-25            NaN -0.001079    0.0  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-23,0.001503,0.002681,NaN,0.000560,0.000510,0.000805,0.002077,0.000000,0.001183,NaN,...,0.000399,0.000633,0.001286,0.0,0.0,0.000088,0.007873,0.000697,0.001240,0.008174
2021-06-24,0.000000,0.000000,NaN,0.000392,0.000510,0.001214,0.001629,0.000363,0.000000,NaN,...,0.000319,0.000652,0.000428,0.0,0.0,0.001144,0.001528,0.001393,0.001857,0.000000
2021-06-25,0.000706,0.000000,NaN,0.001566,0.000509,0.000551,0.000887,0.000121,0.000000,NaN,...,0.000000,0.000555,0.005991,0.0,0.0,0.001318,0.000678,0.000000,0.000865,0.000000


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                         \
Admin2         Autauga Baldwin   Barbour Bibb Blount Bullock Butler   Calhoun   
2021-06-23         0.0     0.0  0.016949  0.0    0.0     0.0    0.0  0.006116   
2021-06-24         0.0     0.0  0.000000  0.0    0.0     0.0    0.0  0.000000   
2021-06-25         0.0     0.0  0.000000  0.0    0.0     0.0    0.0  0.003040   

Province_State                    ... Wyoming                              \
Admin2         Chambers Cherokee  ...    Park    Platte Sheridan Sublette   
2021-06-23          0.0      0.0  ...     0.0  0.090909      0.0      0.0   
2021-06-24          0.0      0.0  ...     0.0  0.000000      0.0      0.0   
2021-06-25          0.0      0.0  ...     0.0  0.000000      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-23            0.0   0.0   0.0        NaN      0.0    0.0  
2021-06-24            0.0   0.0   0.0        NaN      0.0    0.0  
2021-06-25            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-23,0.022126,0.000018,0.002775,0.000532,0.003140,1.188060e-09,0.005113,0.000363,0.000627,0.000157,...,0.002865,0.005158,0.003863,1.807004e-20,0.002530,0.018245,0.000261,0.000718,0.022178,0.014003
2021-06-24,0.018357,0.000017,0.002679,0.000410,0.002741,5.940301e-10,0.005384,0.000411,0.000642,0.000138,...,0.003216,0.005004,0.003992,9.035018e-21,0.005574,0.021060,0.000130,0.000866,0.024203,0.016500
2021-06-25,0.018582,0.000035,0.002682,0.000385,0.005046,2.970151e-10,0.005453,0.000433,0.000863,0.000141,...,0.003243,0.004693,0.004206,4.517509e-21,0.005253,0.018001,0.000872,0.000433,0.021329,0.018531


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-23,0.022848,0.000103,0.002609,1.835871e-18,0.005492,1.020598e-12,0.007313,0.000369,1.164787e-25,0.000208,...,0.000141,0.005748,0.002255,0.0,0.002700,0.016230,0.000233,0.000133,0.029160,0.002703
2021-06-24,0.021273,0.000051,0.002534,9.179353e-19,0.004455,5.102989e-13,0.006140,0.000407,5.823934e-26,0.000619,...,0.000152,0.006502,0.001820,0.0,0.007037,0.022401,0.000117,0.000435,0.031581,0.006375
2021-06-25,0.018161,0.000026,0.002493,4.589676e-19,0.003362,2.551495e-13,0.006028,0.000203,2.911967e-26,0.000450,...,0.000146,0.006392,0.002293,0.0,0.006000,0.025089,0.000199,0.000586,0.031963,0.006698


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-23,0.008300,0.000106,0.002642,0.001000,0.007653,0.000045,0.004571,0.000284,0.000140,0.000337,...,0.002964,0.008445,0.004525,4.336809e-19,0.002979,0.026990,0.000312,0.004635,0.013395,0.003254
2021-06-24,0.009422,0.000095,0.002727,0.000719,0.004105,0.000023,0.005240,0.000332,0.000217,0.000312,...,0.001705,0.008527,0.004193,2.168404e-19,0.007934,0.020093,0.000156,0.002944,0.019026,0.002254
2021-06-25,0.007585,0.000082,0.002823,0.000469,0.006150,0.000011,0.005475,0.000422,0.000235,0.000257,...,0.001043,0.008409,0.004682,1.084202e-19,0.006160,0.026542,0.001226,0.002723,0.024669,0.002193


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-23,0.000441,0.000690,NaN,0.000510,0.001017,0.000264,0.000758,0.000125,0.000177,7.146677e-141,...,0.000186,0.000436,0.000925,0.000243,0.000751,0.000258,0.001347,0.000572,0.000160,0.001201
2021-06-24,0.000220,0.000345,NaN,0.000606,0.001061,0.000277,0.000912,0.000282,0.000184,3.573338e-141,...,0.000129,0.000483,0.001022,0.000245,0.000375,0.000316,0.001282,0.000454,0.000182,0.001175
2021-06-25,0.000602,0.000172,NaN,0.000632,0.000966,0.000294,0.000818,0.000323,0.000165,1.786669e-141,...,0.000064,0.000513,0.000885,0.000266,0.008539,0.000431,0.001188,0.000410,0.000140,0.001428


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-06-23      0.000181  0.000671 -0.000219  0.000201  0.000601  0.000615   
2021-06-24      0.000091  0.000336 -0.000110  0.000100  0.000301  0.000308   
2021-06-25      0.000045  0.000715 -0.000055  0.000050  0.000869  0.000555   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-06-23      0.001132  0.000080  0.000416  0.000542  ...  0.000703   
2021-06-24      0.000566  0.000040  0.000208  0.000271  ...  0.001042   
2021-06-25      0.001394  0.001242  0.000372 -0.000131  ...  0.000521   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-06-23      0.000928  0.001314 -0.001869   0.002095  0.000034 -0.000373   
2021-06-24      0.001941  0.001117 -0.002186   0.002214  0.000017  0.000031   
2021-06-25      0.001707  0.001018 -0.001093   0.001742  0.000403  0.002185   

Province_State                                     
Admin2         Unassigned  Washakie        Weston  
2021-06-23           -1.0  0.000272  2.349249e-08  
2021-06-24           -1.0  0.000136  1.174625e-08  
2021-06-25           -1.0 -0.000471  5.873124e-09  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-23,0.000836,0.001342,NaN,0.000559,0.000655,0.000564,0.001350,0.000055,0.001873,NaN,...,0.000493,0.000535,0.001061,3.740015e-09,0.000204,0.000361,0.004578,0.000976,0.000995,0.004112
2021-06-24,0.000418,0.000671,NaN,0.000476,0.000582,0.000889,0.001489,0.000209,0.000936,NaN,...,0.000406,0.000593,0.000745,1.870008e-09,0.000102,0.000752,0.003053,0.001184,0.001426,0.002056
2021-06-25,0.000562,0.000335,NaN,0.001021,0.000546,0.000720,0.001188,0.000165,0.000468,NaN,...,0.000203,0.000574,0.003368,9.350038e-10,0.000051,0.001035,0.001866,0.000592,0.001146,0.001028


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                                  \
Admin2               Autauga   Baldwin   Barbour          Bibb        Blount   
2021-06-23      7.779631e-08  0.000050  0.008475  5.650609e-17  6.671455e-15   
2021-06-24      3.889816e-08  0.000025  0.004237  2.825305e-17  3.335727e-15   
2021-06-25      1.944908e-08  0.000013  0.002119  1.412652e-17  1.667864e-15   

Province_State                                                                \
Admin2               Bullock        Butler   Calhoun  Chambers      Cherokee   
2021-06-23      2.218419e-14  5.490561e-11  0.003154  0.000031  6.166395e-22   
2021-06-24      1.109209e-14  2.745281e-11  0.001577  0.000015  3.083198e-22   
2021-06-25      5.546046e-15  1.372640e-11  0.002308  0.000008  1.541599e-22   

Province_State  ...       Wyoming                                        \
Admin2          ...          Park    Platte      Sheridan      Sublette   
2021-06-23      ...  3.482327e-30  0.045455  2.630071e-32  1.155414e-52   
2021-06-24      ...  1.741164e-30  0.022727  1.315036e-32  5.777070e-53   
2021-06-25      ...  8.705818e-31  0.011364  6.575178e-33  2.888535e-53   

Province_State                                                              \
Admin2         Sweetwater     Teton         Uinta Unassigned      Washakie   
2021-06-23       0.000050  0.000007  3.700743e-17   0.752941  7.178782e-45   
2021-06-24       0.000025  0.000003  1.850372e-17   0.752941  3.589391e-45   
2021-06-25       0.000013  0.000002  9.251859e-18   0.752941  1.794695e-45   

Province_State            
Admin2            Weston  
2021-06-23      0.000006  
2021-06-24      0.000003  
2021-06-25      0.000002  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
